In [ ]:
import os, sys
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings('ignore')

from files.functions import (
    dataSetup,
    fullDataPath,
    _arima_grid_search,
    _standardized_rmse,
    _save_model_artifact,
    _save_validation_predictions,
    _save_future_predictions,
    _save_metrics,
    generate_future_index,
)
from files.CONSTANTS import COIN, RESPONSE_VARIABLE, TRAINING_COLUMNS, TEST_DAYS, TRAIN_PCT
from implementations.arima_model import ARIMAModel

# ARIMA Training

Univariate AutoRegressive Integrated Moving Average (ARIMA) model for crypto price prediction.
This notebook uses only the price/response series and tunes the `(p, d, q)` order by minimizing AIC
on the training split. Rolling one-step-ahead predictions are used for validation.
The final model is re-trained on the full series before forecasting future prices.

In [ ]:
# ── Step 1: Load Data ────────────────────────────────────────────────────────
raw_path = fullDataPath(COIN)
data = pd.read_csv(raw_path)
daily_data = dataSetup(data, trainingColPath=TRAINING_COLUMNS, response=RESPONSE_VARIABLE)

# Extract univariate price series
price_series = daily_data[RESPONSE_VARIABLE].copy()
print(f"Dataset shape: {daily_data.shape}")
print(f"Price series length: {len(price_series)}")
print(f"Date range: {price_series.index[0]} → {price_series.index[-1]}")

price_series.plot(title=f"{COIN} Close Price", figsize=(12, 4))
plt.ylabel("Price (USD)")
plt.tight_layout()
plt.show()

# ── Step 2: Train / Validation Split ─────────────────────────────────────────
n = len(price_series)
split_idx = max(1, int(n * TRAIN_PCT))
train_series = price_series.iloc[:split_idx]
val_series = price_series.iloc[split_idx:]
print(f"\nTrain size: {len(train_series)}  |  Val size: {len(val_series)}")

In [ ]:
# ── Step 3: Define Hyperparameter Grid ───────────────────────────────────────
# ARIMA(p, d, q): p = AR order, d = integration order, q = MA order
p_range = (0, 1, 2)   # AR lags to try
d_range = (0, 1)      # differencing orders to try
q_range = (0, 1, 2)   # MA lags to try

total_combos = len(p_range) * len(d_range) * len(q_range)
print(f"p_range: {p_range}")
print(f"d_range: {d_range}")
print(f"q_range: {q_range}")
print(f"Total (p,d,q) combinations: {total_combos}")

In [ ]:
# ── Step 4: Tune Hyperparameters — Grid Search by AIC ────────────────────────
print("Running ARIMA grid search (AIC-based)...")
best_order, tuning_df = _arima_grid_search(
    train_series, p_range=p_range, d_range=d_range, q_range=q_range
)

print(f"\nBest ARIMA order: {best_order}")

# Show top-5 results by AIC
print("\nTop-5 configurations by AIC:")
if not tuning_df.empty:
    print(tuning_df.head(5).to_string(index=False))

In [ ]:
# ── Step 5: Train Best Model on Training Series ───────────────────────────────
p, d, q = best_order
model = ARIMAModel(p, d, q)
model.fit(train_series)
print(f"ARIMA{best_order} fitted on training series ({len(train_series)} obs).")
print(f"AIC: {model.aic:.2f}  |  BIC: {model.bic:.2f}")

# Show model summary
print("\nModel Summary:")
print(model.summary())

In [ ]:
# ── Step 6: Save Model to models/{COIN}/ ─────────────────────────────────────
# First retrain on FULL series so saved model is ready for production forecasting
final_model = ARIMAModel(*best_order)
final_model.fit(price_series)
print(f"Final ARIMA{best_order} trained on full series ({len(price_series)} obs).")

artifact_path = _save_model_artifact(final_model, COIN, f'{COIN}_arima_model.pkl')
print(f"Model saved to: {artifact_path}")

In [ ]:
# ── Step 7: Predict on Validation Set (rolling one-step-ahead) ───────────────
from statsmodels.tsa.arima.model import ARIMA as _ARIMA

print(f"Running rolling one-step forecasts over {len(val_series)} validation points...")
val_preds_list = []
history = train_series.tolist()
for actual in val_series:
    tmp = _ARIMA(history, order=best_order).fit()
    val_preds_list.append(float(tmp.forecast(steps=1)[0]))
    history.append(actual)

val_preds = np.array(val_preds_list)
val_df = pd.DataFrame({'predicted_price': val_preds}, index=val_series.index)

# Compute RMSE
rmse, std_rmse = _standardized_rmse(val_series, val_preds)
print(f"\nValidation RMSE:            {rmse:,.2f}")
print(f"Standardized RMSE (÷ std):  {std_rmse:.4f}")

# Plot actual vs predicted
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(val_series.index, val_series.values, label='Actual', linewidth=2)
ax.plot(val_df.index, val_df['predicted_price'], label='Predicted (ARIMA)', linestyle='--')
ax.set_title(f"{COIN} ARIMA{best_order} — Validation Set: Actual vs Predicted")
ax.set_xlabel("Date")
ax.set_ylabel("Price (USD)")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# ── Step 8: Predict Next TEST_DAYS Days ──────────────────────────────────────
# final_model was already trained on the full price_series (Step 6)
forecast_vals = final_model.forecast(steps=TEST_DAYS)
future_index = generate_future_index(daily_data.index[-1], TEST_DAYS)
future_df = pd.DataFrame({'predicted_price': forecast_vals}, index=future_index)

print(f"Future predictions for next {TEST_DAYS} days:")
print(future_df.to_string())

# Plot future forecast
fig, ax = plt.subplots(figsize=(12, 5))
hist = price_series.iloc[-60:]
ax.plot(hist.index, hist.values, label='Historical', linewidth=2)
ax.plot(future_df.index, future_df['predicted_price'], marker='o',
        linestyle='--', label=f'ARIMA Forecast ({TEST_DAYS}d)', color='orange')
ax.set_title(f"{COIN} ARIMA{best_order} — Next {TEST_DAYS}-Day Price Forecast")
ax.set_xlabel("Date")
ax.set_ylabel("Price (USD)")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# ── Step 9: Save Predictions and Metrics ─────────────────────────────────────
val_path = _save_validation_predictions(val_df, COIN, 'arima')
future_path = _save_future_predictions(future_df, COIN, 'arima')
metrics_path = _save_metrics(std_rmse, COIN, 'arima')

print(f"Validation predictions saved to: {val_path}")
print(f"Future predictions saved to:     {future_path}")
print(f"Metrics (std RMSE) saved to:     {metrics_path}")
print(f"\nStandardized RMSE: {std_rmse:.4f}")
print("Done.")